In [2]:
import re
import os
import json
import pandas as pd

In [3]:

data = pd.read_csv("data_processed/test.csv")
data_filtered = data[data["hasDetails"]]
print(data)
print(data_filtered)

              id                                                url   price  \
0       58155817  /mercedes-benz-clase-c-c-cabrio-220-d-2p-diese...   32499   
1       58594130  /mazda-mx-5-active-18-2p-gasolina-2006-en-ciud...   10600   
2       58548465  /kia-sorento-diesel-2005-en-malaga-58548465-co...    4500   
3       58594037  /opel-cabrio-14-t-ss-excellence-2p-gasolina-20...   12450   
4       58707917  /porsche-911-turbo-coupe-2p-gasolina-2016-en-b...  134900   
...          ...                                                ...     ...   
269026  58568453  /bmw-serie-3-325d-2p-diesel-2008-en-alicante-5...   12000   
269027  58747631  /nissan-qashqai-digt-103kw-140cv-mhev-4x2-acen...   21500   
269028  58779068  /audi-a3-sportback-genuine-40-tfsi-e-150kw-s-t...   38200   
269029  58778999  /citroen-c3-11i-sx-5p-gasolina-2005-en-madrid-...    2995   
269030  58758435  /citroen-c4-14-16v-collection-5p-gasolina-2008...    2999   

              km    year         color  cubicCapaci